在這個 notebook 中我們會介紹 MobileNet v1 的 Architecture Design

# Architecture Design

## Depthwise & Pointwise Convolution

<img src="hw7_data/Depthwise & Pointwise Convolution.png">

> 藍色為上下層 Channel 的關係，綠色則為該 Receptive Field 的擴張。

(a) 就是一般的 Convolution Layer，所以他的 Weight 連接方式會跟 Fully Connected 一樣，只差在原本在 Fully Connected 是用數字相乘後相加，Convolution Layer 是圖片卷積後相加。

(b) DW (Depthwise Convolution Layer)你可以想像成一張 feature map 各自過**一個 filter** 處理後，再用 PW(Pointwise Convolution Layer)把所有 feature map 的單個 pixel 資訊合在一起(就是1個 pixel 的 Fully Connected Layer)。

(c) GC(Group Convolution Layer)就是把 feature map 分組，讓他們自己過 Convolution Layer 後再重新 Concat 起來。算是一般的 Convolution 和 Depthwise Convolution 的折衷版。**所以說，Group Convolution的Group = Input Feautures數就會是Depthwise Convolution(因為每個Channel都各自獨立)，Group = 1就會是一般的Convolution(因為就等於沒有Group)。**

| | |
|:-|:-|
| <img src="hw7_data/Depthwise Convolution.jpeg" width="60%"> | Depthwise Convolution |
| <img src="hw7_data/Pointwise Convolution.jpeg" width="60%"> | Pointwise Convolution |
| <img src="hw7_data/Group Convolution.png" width="60%" > | Group Convolution |


## 實作細節

```python
# 一般的 Convolution, weight大小 = in_chs * out_chs * kernel_size^2
nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding) 

# Group Convolution, Group 數目可以自行控制，表示要分成幾群。其中in_chs和out_chs必須要可以被groups整除。(不然沒辦法分群。)
nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding, groups = groups)

# Depthwise Convolution, 輸入chs = 輸出chs = Groups數目, weight大小 = in_chs * kernel_size^2
nn.Conv2d(in_chs, out_chs = in_chs, kernel_size, stride, padding, groups = in_chs)

# Pointwise Convolution, 也就是 1 by 1 convolution, weight大小 = in_chs * out_chs
nn.Conv2d(in_chs, out_chs, 1)
```

# Model

training 的部分請參考Network Pruning、Knowledge Distillation，或直接只用hw3的手把手即可。

> 註記: 這邊把各個Block多用一層 Sequential 包起來是因為 Network Pruning 的時候抓 Layer 比較方便。

在這個Net裡面，我們會使用Depthwise + Pointwise Convolution Layer來疊model
- 先单独处理每一个 in_channel ，一个 in_channel 得到一个矩阵，矩阵的个数为 in_channel 的个数
- 再将所有得到的矩阵通过**一个** size 为 (1 * 1 * in_channel) 的 filter 组合成一个矩阵，通过设置多个 filter ，就可以得到多个 out_channel

你會發現，將原本的Convolution Layer換成Dw & Pw後，Accuracy通常不會降很多。另外，取名為StudentNet是因為這個Model等會要做Knowledge Distillation。

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class StudentNet(nn.Module):
    def __init__(self, base = 16, width_mult = 1): # width_mult 控制每层 filter 的个数，base=16
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 16, 16]
        bandwidth = [ base * m for m in multiplier] # bandwidth: 每一層Layer所使用的 channel 數量，都是16的倍数
        for i in range(3, 7): # 我們只 Pruning 第三層以後的Layer
            bandwidth[i] = int(bandwidth[i] * width_mult) # width_mult 控制每层 filter 的个数

        self.cnn = nn.Sequential( # 第一層我們通常不會拆解Convolution Layer。
            nn.Sequential(
                nn.Conv2d(3, bandwidth[0], 3, 1, 1), # [in_channels, out_channels, kernel_size, stride, padding] 
                nn.BatchNorm2d(bandwidth[0]),
                nn.ReLU6(),
                nn.MaxPool2d(2, 2, 0), # kernel_size, stride, padding
            ),
            nn.Sequential( # 接下來每一個Sequential Block都一樣，所以我們只講一個Block
                nn.Conv2d(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]), # Depthwise Convolution，每层单独算
                nn.BatchNorm2d(bandwidth[0]), # Batch Normalization
                nn.ReLU6(), # ReLU6 限制 Neuron 在[0, 6]， MobileNet 系列都是使用ReLU6, 因为如果數字太大，會不好壓到float16 / or further qunatization
                nn.Conv2d(bandwidth[0], bandwidth[1], 1), # Pointwise Convolution 层数 bandwidth[0] 变成 bandwidth[1]
                nn.MaxPool2d(2, 2, 0),# 過完Pointwise Convolution不需要再做ReLU，經驗上Pointwise + ReLU效果都會變差
                # 每過完一個Block就Down Sampling
            ),
            nn.Sequential(
                nn.Conv2d(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                nn.BatchNorm2d(bandwidth[1]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[1], bandwidth[2], 1),
                nn.MaxPool2d(2, 2, 0),
            ),
            nn.Sequential(
                nn.Conv2d(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                nn.BatchNorm2d(bandwidth[2]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[2], bandwidth[3], 1),
                nn.MaxPool2d(2, 2, 0),
            ),
            nn.Sequential( # 到這邊為止因為圖片已經被Down Sample很多次了，所以就不做MaxPool
                nn.Conv2d(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                nn.BatchNorm2d(bandwidth[3]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[3], bandwidth[4], 1),
            ),
            nn.Sequential(
                nn.Conv2d(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                nn.BatchNorm2d(bandwidth[4]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[4], bandwidth[5], 1),
            ),
            nn.Sequential(
                nn.Conv2d(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                nn.BatchNorm2d(bandwidth[5]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[5], bandwidth[6], 1),
            ),
            nn.Sequential(
                nn.Conv2d(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                nn.BatchNorm2d(bandwidth[6]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[6], bandwidth[7], 1),
            ),
            # 這邊我們採用Global Average Pooling。
            # 如果輸入圖片大小不一樣的話，就會因為Global Average Pooling壓成一樣的形狀，這樣子接下來做FC就不會對不起來。
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.fc = nn.Sequential(nn.Linear(bandwidth[7], 11)) # 這邊我們直接Project到11維輸出答案。

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
